# Proyecto individual ML_Ops Movies

In [140]:
import pandas as pd
from pandas import json_normalize
import numpy as np
from datetime import datetime
import ast  # Importar el módulo ast

Hacemos una interseccion con los dos dataframe df_movies y df_credits

In [141]:
df_credits = pd.read_csv("c:\\Users\\Giuliano\\Desktop\\credits.csv")
df_movies = pd.read_csv("c:\\Users\\Giuliano\\Desktop\\movies_dataset.csv",low_memory=False)

In [142]:
# Convertir la columna 'id' en ambos DataFrames a tipo string
df_movies['id'] = df_movies['id'].astype(str)
df_credits['id'] = df_credits['id'].astype(str)

In [143]:
# Merge entre df_movies y df_credits usando la columna 'id' en df_movies y 'id' en df_credits.
df = pd.merge(df_movies, df_credits, left_on='id', right_on='id', how='inner')

ETL

In [144]:
# Cambiar el nombre de la columna 'id' en df a 'movie_id'
df.rename(columns={'id': 'movie_id'}, inplace=True)

Desanidamos genres

In [145]:
df_genres = pd.DataFrame(df['genres'])

# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_genres["genres"] = df_genres['genres'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_genres = df_genres.explode('genres')
# Convertir los diccionarios en columnas separadas
df_genres = pd.json_normalize(df_genres['genres'])

df_genres.rename(columns={'id': 'genres_id',"name":"Genero"}, inplace=True)


In [146]:
# Unir el DataFrame desanidado con el DataFrame original
df= pd.concat([df, df_genres], axis=1)

Desanidamos belongs_to_collection

In [147]:
# Convertir las cadenas en diccionarios
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)


In [148]:
df_desanidado = pd.json_normalize(df['belongs_to_collection'])

In [149]:
# Unir el DataFrame desanidado con el DataFrame original
df= pd.concat([df, df_desanidado], axis=1)

In [150]:
# Cambiar el nombre de la columna 'id' en df_credits a 'movie_id'
df.rename(columns={'id': 'franquicia_id'}, inplace=True)
# Eliminar la columna ''
df.drop(columns=['poster_path'], inplace=True)

Desanidamos product_companies

In [151]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df['production_companies'] = df['production_companies'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_expanded = df.explode('production_companies')
# Convertir los diccionarios en columnas separadas
df_companies = pd.json_normalize(df_expanded['production_companies'])

In [152]:
df = pd.concat([df,df_companies], axis=1)

In [153]:
# Borramos las columnas bellong_to_collection y product_companies
df = df.drop(columns=['belongs_to_collection', 'production_companies'])

In [154]:
# Reemplazar los valores nulos en las columnas 'revenue' y 'budget' por 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df_movies['budget'].fillna(0)

In [155]:
# Eliminar las filas donde el valor de 'release_date' es nulo
df = df.dropna(subset=['release_date'])

In [156]:
# Verificar cuántos valores nulos quedan en 'release_date'
nulos_release_date = df['release_date'].isnull().sum()
nulos_release_date

np.int64(0)

In [157]:
# Asegurarse de que 'release_date' esté en formato de fecha AAAA-mm-dd
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d',errors="coerce")

In [158]:
# Eliminar las filas donde 'release_date' no pudo convertirse (valores nulos)
df = df.dropna(subset=['release_date'])

In [159]:
# Crear una nueva columna 'release_year' extrayendo el año de 'release_date'
df['release_year'] = df['release_date'].dt.year

In [160]:
# Asegurarse de que 'revenue' y 'budget' sean numéricos, reemplazar valores no convertibles por 0
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').fillna(0)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').fillna(0)

In [161]:
df["retorno_de_inversion"] = df["revenue"] / df["budget"] 

In [162]:
df["retorno_de_inversion"] = df["retorno_de_inversion"].fillna(0)

In [163]:
df["retorno_de_inversion"] = df["retorno_de_inversion"][
        (df['retorno_de_inversion'] != np.inf) &  # Excluir infinito
        (df['retorno_de_inversion'] != -np.inf) &  # Excluir -infinito
        (df['retorno_de_inversion'].notna()) &  # Excluir NaN
        (df['retorno_de_inversion'] >= 0)  # Asegurarse de que el valor sea positivo
    ]

In [164]:
# Si está correcto, elimina las columnas
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'homepage',
        'original_language', 'original_title', 'overview',
       'production_countries', 'runtime',
       'spoken_languages', 'status', 'tagline', 'video', 'genres_id',
       'franquicia_id', 'backdrop_path'])

Desanidamos cast y crew

In [165]:
df_cast = pd.DataFrame(df['cast'])

In [166]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_cast['cast'] = df_cast['cast'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

In [167]:
# Expandir las listas en filas separadas
df_cast = df_cast.explode('cast')

In [168]:
# Convertir los diccionarios en columnas separadas
df_cast = pd.json_normalize(df_cast['cast'])

In [169]:
df_cast.rename(columns={'id': 'actor_id'}, inplace=True)

In [170]:
df.columns

Index(['budget', 'genres', 'movie_id', 'popularity', 'release_date', 'revenue',
       'title', 'vote_average', 'vote_count', 'cast', 'crew', 'Genero', 'name',
       'name', 'id', 'release_year', 'retorno_de_inversion'],
      dtype='object')

In [171]:
df_crew = pd.DataFrame(df['crew'])

In [172]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_crew['crew'] = df_crew['crew'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_crew = df_crew.explode('crew')

# Convertir los diccionarios en columnas separadas
df_crew = pd.json_normalize(df_crew['crew'])


In [173]:
df = pd.concat([df,df_crew],axis=1)

In [174]:
df = pd.concat([df,df_cast],axis=1)

In [175]:
# Definir las columnas que deseas eliminar
columns_to_drop = ["genres",'cast', 'crew', 'id', 'credit_id', 'department','gender', 'id', 'job', 'profile_path', 'cast_id', 'character', 'credit_id', 'gender', 'actor_id', 'order', 'profile_path']

# Verificar cuáles de esas columnas realmente existen en el dataframe
columns_present = [col for col in columns_to_drop if col in df.columns]


# Eliminar solo las columnas que existen
df = df.drop(columns=columns_present)

In [181]:
 #Renombramos columnas
 df.columns = ['budget', 'movie_id', 'popularity', 'release_date', 'revenue', 'title',
       'vote_average', 'vote_count', 'Genero', 'name', 'company', 'release_year',
       'retorno_de_inversion', 'director', 'actor']

In [183]:
df.to_csv("df_movies.csv")